**Finetuning LLM Model (Llama-3.2-3B-Instruct) using Lora technique**

In [1]:
pip install unsloth transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.7/348.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.6/273.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: torchao
    Found existing installation: torchao 0.10.0
    Uninstalling torchao-0.10.0:
      

**unsloth** - speed up fine tuning and reduces gpu usage for llma , mistral ,gemma models(Supports Llora , Qllora ,Quanti)

**TRL**- Transformers Reinforment learning (HF Librart)


In [2]:
import torch
from unsloth import FastLanguageModel # unsloth - speed up fine tuning and reduces gpu usage for llma , mistral ,gemma models(Supports Llora , Qllora ,Quanti)
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
model , tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=2048,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model , r =16 ,
    target_modules = ["q_proj" ,  "k_proj", "v_proj", "o_proj" ,  "gate_proj", "up_proj", "down_proj"]
)

Unsloth 2025.10.12 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
#q - query , k - key , v - value , o - output , gate -

In [6]:
tokenizer = get_chat_template(tokenizer , chat_template="llama-3.1") # must for every case and model scenario

In [7]:
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [8]:
dataset = standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [9]:
dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [10]:
dataset[1]

{'conversations': [{'content': 'Explain how recursion works and provide a recursive function in Python that calculates the factorial of a given number.',
   'role': 'user'},
  {'content': "Recursion is a programming technique where a function calls itself to solve a problem. It breaks down a complex problem into smaller, more manageable subproblems until a base case is reached. The base case is a condition where the function does not call itself, but instead returns a specific value or performs a specific action.\n\nIn the case of calculating the factorial of a number, recursion can be used to break down the problem into simpler subproblems. The factorial of a non-negative integer n is the product of all positive integers less than or equal to n.\n\nHere is a recursive function in Python that calculates the factorial of a given number:\n\n```python\ndef factorial(n):\n    # Base case: factorial of 0 or 1 is 1\n    if n == 0 or n == 1:\n        return 1\n    # Recursive case: factorial 

In [11]:
dataset = dataset.map(
    lambda examples:{
        "text" :[
            tokenizer.apply_chat_template(convo, tokenize=False)
            for convo in examples["conversations"]
        ]
    },
    batched = True
)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [12]:
dataset

Dataset({
    features: ['conversations', 'source', 'score', 'text'],
    num_rows: 100000
})

In [13]:
dataset[1]

{'conversations': [{'content': 'Explain how recursion works and provide a recursive function in Python that calculates the factorial of a given number.',
   'role': 'user'},
  {'content': "Recursion is a programming technique where a function calls itself to solve a problem. It breaks down a complex problem into smaller, more manageable subproblems until a base case is reached. The base case is a condition where the function does not call itself, but instead returns a specific value or performs a specific action.\n\nIn the case of calculating the factorial of a number, recursion can be used to break down the problem into simpler subproblems. The factorial of a non-negative integer n is the product of all positive integers less than or equal to n.\n\nHere is a recursive function in Python that calculates the factorial of a given number:\n\n```python\ndef factorial(n):\n    # Base case: factorial of 0 or 1 is 1\n    if n == 0 or n == 1:\n        return 1\n    # Recursive case: factorial 

In [14]:
trainer = SFTTrainer(
    model=model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    tokenizer=tokenizer, # Pass the tokenizer explicitly
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 6,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "output"
        # optim = "paged_adamw_8bit"
    ),

)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [15]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shinde-ashubham (shinde-ashubham-glib) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.439900
2,1.846100
3,1.383900
4,1.425100
5,1.364900
6,1.466600
7,0.976400
8,1.528500
9,1.255700
10,1.270200


TrainOutput(global_step=60, training_loss=1.030725791056951, metrics={'train_runtime': 548.5592, 'train_samples_per_second': 0.875, 'train_steps_per_second': 0.109, 'total_flos': 5495762487078912.0, 'train_loss': 1.030725791056951, 'epoch': 0.0048})

In [16]:
model.save_pretrained("finetuned_model")

In [17]:
inference_model , inference_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./finetuned_model",
    max_seq_length=2048,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [18]:
text_prompts = [
    "what are rules in cricket"
]
for prompt in text_prompts:
  formatted_prompt = inference_tokenizer.apply_chat_template([{
      "role": "user",
      "content": prompt
      }], tokenize=False)

  model_inputs = inference_tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
  generated_ids = inference_model.generate(
      **model_inputs,
      max_new_tokens=512,
      temperature=0.7,
      do_sample=True,
      pad_token_id=inference_tokenizer.pad_token_id
  )
  response = inference_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 01 Nov 2025

user

what are rules in cricketassistant

In cricket, there are several rules that govern the game. Here are some of the most important ones:

1. The Objective: The objective of cricket is to score runs by hitting the ball with a bat and running between the wickets, while the opposing team tries to stop them by getting the batsmen out.

2. The Equipment: Cricket is played with a ball, a bat, and a set of wickets (three vertical stumps and two horizontal sticks). The ball is made of cork, wrapped in yarn, and covered with a leather cover. The bat is made of wood or composite materials, and the wickets are made of wood.

3. The Game Structure: A cricket match is divided into innings, with each team taking turns to bat and bowl. The batting team sends two batsmen onto the field, and the bowling team sends two bowlers. The batting team tries to score as many runs as possible while the opposing team tries to get them out

In [19]:
history = []  # store conversation here

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    # Add user message to history
    history.append({"role": "user", "content": user_input})

    # Apply chat template to full history
    formatted_prompt = inference_tokenizer.apply_chat_template(
        history, tokenize=False
    )

    model_inputs = inference_tokenizer(
        formatted_prompt,
        return_tensors="pt"
    ).to("cuda")

    # Generate model response
    generated_ids = inference_model.generate(
        **model_inputs,
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
        pad_token_id=inference_tokenizer.pad_token_id
    )

    response = inference_tokenizer.decode(
        generated_ids[0], skip_special_tokens=True
    )

    # Extract only the latest assistant response
    assistant_reply = response.split("assistant")[-1].strip()

    print("Assistant:", assistant_reply)

    # Add model response to history
    history.append({"role": "assistant", "content": assistant_reply})


You: what is sun
Assistant: The sun is a star at the center of our solar system. It is the primary source of light, heat, and energy for Earth and other planets. The sun is a massive ball of hot, glowing gas, primarily composed of hydrogen and helium. It is about 93 million miles (150 million kilometers) away from Earth, and it takes about 8 minutes and 20 seconds for light from the sun to reach our planet. The sun is about 93 million miles (150 million kilometers) away from Earth, and it takes about 8 minutes and 20 seconds for light from the sun to reach our planet.
You: who is elon
Assistant: Elon Musk is a South African-born entrepreneur, inventor, and business magnate. He is the founder and CEO of SpaceX and Tesla, Inc., and is known for his innovative and ambitious projects in the fields of electric cars, renewable energy, and space exploration. Musk has been instrumental in popularizing the electric vehicle and clean energy industries, and has been a driving force behind the dev